# Processing

This notebook reads in the 2007 and 2017 reports from the U.S. Department's Federal Highway Administration and processes them into cleaned data files for analysis (see [`02_analysis.ipynb`](https://github.com/baltimore-sun-data/bridge-data/blob/master/02_analysis.ipynb).

## Import R libraries

In [1]:
suppressMessages(library('tidyverse'))
suppressMessages(library('readxl'))
suppressMessages(library('feather'))

## Read in Maryland data from 2007 and 2017

The Maryland reports are downloaded from the National Bridge Inventory, which is collected by the U.S. Department of Transportation's Federal Highway Administration: https://www.fhwa.dot.gov/bridge/nbi/ascii2007.cfm (2007) and https://www.fhwa.dot.gov/bridge/nbi/ascii2017.cfm (2017)

In [2]:
data.2007 <- suppressMessages(read_csv("input/24fluna_992007-20160909121509.txt", guess_max = 7000))
data.2017 <- suppressMessages(read_csv("input/MD17.txt", guess_max = 7000))

There are more than 100 columns in each report, outlined on the FHA's website https://www.fhwa.dot.gov/bridge/nbi/format.cfm as well as in this PDF file https://www.fhwa.dot.gov/bridge/mtguide.pdf. Each bridge is uniquely identified by a 15-digit ID column, `STRUCTURE_NUMBER_008`.

## Read in fips codes to geographically identify counties

Census fips codes for states, counties, places, via https://www.census.gov/geographies/reference-files/2017/demo/popest/2017-fips.html

In [3]:
fips <- read_excel("input/all-geocodes-v2017.xlsx", skip = 3) 
fips$combinedfips <- paste0(fips$`State Code (FIPS)`, fips$`County Code (FIPS)`)
fips$combinedplacefips <- paste0(fips$`State Code (FIPS)`, fips$`Place Code (FIPS)`)
fips <- fips %>% filter(`County Subdivision Code (FIPS)` == '00000')

## Process data

The below function processes the data for each year by renaming and recoding a subset of relevant columns and creating the indicator column `bridge_condition` which is either "Poor", "Fair", or "Good" and defined by the Federal Highway Administration as follows https://www.fhwa.dot.gov/bridge/britab.cfm: 

- "Bridge Condition is determined by the lowest rating of National Bridge Inventory (NBI) condition ratings for Item 58 (Deck), Item 59 (Superstructure), Item 60 (Substructure), or Item 62 (Culvert). If the lowest rating is greater than or equal to 7, the bridge is classified as Good; if it is less than or equal to 4, the classification is Poor. Bridges rated 5 or 6 are classified as Fair"

The below function also creates an indicator variable, `struct_deficient` which = 1 if the bridge is classified as "structurally deficient" and 0 if not. FHA is transitioning from labeling bridges as "structurally deficient" to this Good-Fair-Poor system. Note that the formula for the "structurally deficient" classification is equivalent to the formula for the "Poor" classification https://www.fhwa.dot.gov/bridge/britab.cfm:

- "A bridge which has a condition rating of 4 or less (Poor or worse condition) for Item 58 (Deck), Item 59 (Superstructure), Item 60 (Substructure), or Item 62 (Culvert)"

In previous years, "structurally deficient" was defined in a slightly different way. This is reflected by the column `struct_deficient_old`:

- "Structurally deficient" was previously defined in https://www.fhwa.dot.gov/bridge/0650dsup.cfm as having a condition rating of 4 or less for Item 58 (Deck), Item 59 (Superstructure), Item 60 (Substructure), or Item 62 (Culvert), OR having an appraisal rating of 2 or less for Item 67 (Structural Condition) or Item 71 (Waterway Adequacy)"

In [1]:
process <- function (df) {

    df <- df %>% rename(avg_daily_traffic = ADT_029, # number of vehicles
                        avg_daily_traffic_yr = YEAR_ADT_030,
                        avg_daily_truck_traffic_perc = PERCENT_ADT_TRUCK_109,
                        yr_built = YEAR_BUILT_027, 
                        inspection_freq = INSPECT_FREQ_MONTHS_091, # designated insepction frequency in months
                        bridge_improve_cost = BRIDGE_IMP_COST_094, # estimated bridge improvement cost in thousands of dollars
                        roadway_improve_cost = ROADWAY_IMP_COST_095, # estimated roadway improvement cost in thousands of dollars
                        total_improve_cost = TOTAL_IMP_COST_096, # estimated total improvement cost in thousands of dollars
                        yr_cost_estimate = YEAR_OF_IMP_097, # year of improvement cost estimate (should be no more than 8 years old)
                        yr_reconstructed = YEAR_RECONSTRUCTED_106, # 0000 for no reconstruction
                        avg_daily_traffic_future = FUTURE_ADT_114, 
                        avg_daily_traffic_future_yr = YEAR_OF_FUTURE_ADT_115, # should be at least 17 years but no more than 22 years from the year of inspection
                        location = LOCATION_009,
                        intersects = FEATURES_DESC_006A,
                        carries = FACILITY_CARRIED_007,
                        route_number = ROUTE_NUMBER_005D) %>% # all zeros for no route 
    
                 mutate(deck_condition = as.numeric(DECK_COND_058), 
                        superstruct_condition = as.numeric(SUPERSTRUCTURE_COND_059),
                        substruct_condition = as.numeric(SUBSTRUCTURE_COND_060),
                        culvert_condition = as.numeric(CULVERT_COND_062), 
                        structural_eval = as.numeric(STRUCTURAL_EVAL_067), 
                        waterway_eval = as.numeric(WATERWAY_EVAL_071),
                        combinedfips = paste0(STATE_CODE_001, COUNTY_CODE_003)) %>%
    
    # SD: indicator for structurally deficient bridges
                 mutate(struct_deficient = case_when(deck_condition <= 4 | 
                                           superstruct_condition <= 4 | 
                                           substruct_condition <= 4 | 
                                           culvert_condition <= 4  ~ 1, 
                                           TRUE ~ 0),
                        
                        struct_deficient_old = case_when(deck_condition <= 4 | 
                                           superstruct_condition <= 4 | 
                                           substruct_condition <= 4 | 
                                           culvert_condition <= 4 |
                                           structural_eval <= 2 | 
                                           waterway_eval <= 2 ~ 1, 
                                           TRUE ~ 0),
                            
    # indicator for each of the conditions: is it poor or worse?
                        deck_condition_poor = ifelse(deck_condition <= 4, 1, 0), 
                        superstruct_condition_poor = ifelse(superstruct_condition <= 4, 1, 0), 
                        substruct_condition_poor = ifelse(substruct_condition <= 4, 1, 0), 
                        culvert_condition_poor = ifelse(culvert_condition <= 4, 1, 0), 
                        
    # responsibility: who is responsible for maintaining the bridge? Here are the main ones                    
                        responsibility = case_when(MAINTENANCE_021 == '01' ~ 'state highway agency',
                                                  MAINTENANCE_021 == '02' ~ 'county highway agency',
                                                  MAINTENANCE_021 == '03' ~ 'town or township highway agency',
                                                  MAINTENANCE_021 == '04' ~ 'city or municipal highway agency',
                                                  MAINTENANCE_021 == '11' ~ 'state park, forest, or reservation agency',
                                                  MAINTENANCE_021 == '12' ~ 'local park, forest, or reservation agency',
                                                  MAINTENANCE_021 == '21' ~ 'other state agencies',
                                                  MAINTENANCE_021 == '25' ~ 'other local agencies',
                                                  MAINTENANCE_021 == '26' ~ 'private (other than railroad)',
                                                  MAINTENANCE_021 == '27' ~ 'railroad',
                                                  MAINTENANCE_021 == '31' ~ 'state toll authority',
                                                  MAINTENANCE_021 == '32' ~ 'local toll authority',
                                                  MAINTENANCE_021 == '60' ~ 'other federal agencies',
                                                  MAINTENANCE_021 == '61' ~ 'Indian Tribal Government',
                                                  MAINTENANCE_021 == '62' ~ 'Bureau of Indian Affairs',
                                                  MAINTENANCE_021 == '63' ~ 'Bureau of Fish and Wildlife',
                                                  MAINTENANCE_021 == '64' ~ 'U.S. Forest Service',
                                                  MAINTENANCE_021 == '66' ~ 'National Park Service',
                                                  MAINTENANCE_021 == '67' ~ 'Tennessee Valley Authority',
                                                  MAINTENANCE_021 == '68' ~ 'Bureau of Land Management',
                                                  MAINTENANCE_021 == '69' ~ 'Bureau of Reclamation',
                                                  MAINTENANCE_021 == '70' ~ 'Corps of Engineers (Civil)',
                                                  MAINTENANCE_021 == '71' ~ 'Corps of Engineers (Military)',
                                                  MAINTENANCE_021 == '72' ~ 'Air Force',
                                                  MAINTENANCE_021 == '73' ~ 'Navy/Marines',
                                                  MAINTENANCE_021 == '74' ~ 'Army',
                                                  MAINTENANCE_021 == '75' ~ 'NASA',
                                                  MAINTENANCE_021 == '76' ~ 'Metropolitan Washington Airports Service',
                                                  MAINTENANCE_021 == '80' ~ 'Unknown'),
    
    # owner: who is the owner of the bridge? (usually the same as responsibility)
                        owner = case_when(OWNER_022 == '01' ~ 'state highway agency',
                                          OWNER_022 == '02' ~ 'county highway agency',
                                          OWNER_022 == '03' ~ 'town or township highway agency',
                                          OWNER_022 == '04' ~ 'city or municipal highway agency',
                                          OWNER_022 == '11' ~ 'state park, forest, or reservation agency',
                                          OWNER_022 == '12' ~ 'local park, forest, or reservation agency',
                                          OWNER_022 == '21' ~ 'other state agencies',
                                          OWNER_022 == '25' ~ 'other local agencies',
                                          OWNER_022 == '26' ~ 'private (other than railroad)',
                                          OWNER_022 == '27' ~ 'railroad',
                                          OWNER_022 == '31' ~ 'state toll authority',
                                          OWNER_022 == '32' ~ 'local toll authority',
                                          OWNER_022 == '60' ~ 'other federal agencies',
                                          OWNER_022 == '61' ~ 'Indian Tribal Government',
                                          OWNER_022 == '62' ~ 'Bureau of Indian Affairs',
                                          OWNER_022 == '63' ~ 'Bureau of Fish and Wildlife',
                                          OWNER_022 == '64' ~ 'U.S. Forest Service',
                                          OWNER_022 == '66' ~ 'National Park Service',
                                          OWNER_022 == '67' ~ 'Tennessee Valley Authority',
                                          OWNER_022 == '68' ~ 'Bureau of Land Management',
                                          OWNER_022 == '69' ~ 'Bureau of Reclamation',
                                          OWNER_022 == '70' ~ 'Corps of Engineers (Civil)',
                                          OWNER_022 == '71' ~ 'Corps of Engineers (Military)',
                                          OWNER_022 == '72' ~ 'Air Force',
                                          OWNER_022 == '73' ~ 'Navy/Marines',
                                          OWNER_022 == '74' ~ 'Army',
                                          OWNER_022 == '75' ~ 'NASA',
                                          OWNER_022 == '76' ~ 'MEtropolitan Washington Airports Service',
                                          OWNER_022 == '80' ~ 'Unknown'),

    # historical significance: is the bridge on the National Register of Historical Bridges?
                        national_register = case_when(HISTORY_037 == 1 ~ 
                                                      'Bridge is on the National Register of Historic Places',
                                                      HISTORY_037 == 2 ~
                                                      'Bridge is eligible for the National Register of Historic Places',
                                                      HISTORY_037 == 3 ~
                                                      'Bridge is possibly eligible for the National Register of Historic Places or on state/local historic register',
                                                      HISTORY_037 == 4 ~ 
                                                      'Historical significance is not determinable at this time',
                                                      HISTORY_037 == 5 ~ 
                                                      'Bridge is not eligible for the National Register of Historic Places'),
                        
                        op_status = case_when(OPEN_CLOSED_POSTED_041 == 'A' ~ 'Open, no restriction', 
                                              OPEN_CLOSED_POSTED_041 == 'B' ~ 'Open, posting recommended but not legally implemented',
                                              OPEN_CLOSED_POSTED_041 == 'D' ~ 'Open, would be posted or closed except for temp shoring, etc.', 
                                              OPEN_CLOSED_POSTED_041 == 'E' ~ 'Open, temp structure in place while original structure is closed',
                                              OPEN_CLOSED_POSTED_041 == 'G' ~ 'New structure not yet open to traffic',
                                              OPEN_CLOSED_POSTED_041 == 'K' ~ 'Bridge closed to all traffic',
                                              OPEN_CLOSED_POSTED_041 == 'P' ~ 'Posted for load (may include other restrictions such as temp bridges which are load posted)',
                                              OPEN_CLOSED_POSTED_041 == 'R' ~ 'Posted for other load-capacity restriction (speed, number of vehicles on bridge, etc.)')) %>%
    rowwise() %>% mutate(
    # lowest rating of the 4 main components
                        lowest_condition_rating = min(deck_condition, 
                                                      superstruct_condition, 
                                                      substruct_condition, 
                                                      culvert_condition, na.rm = T), 
                        
                        total_poor_conditions = sum(deck_condition_poor, 
                                                    superstruct_condition_poor, 
                                                    substruct_condition_poor, 
                                                    culvert_condition_poor, na.rm = T)) %>%
                                           
    
    # bridge condition: Poor, Fair, or Good?
                  mutate(lowest_condition_rating = ifelse(lowest_condition_rating == Inf, NA, 
                                                          lowest_condition_rating), # in case there are bridges that have no condition ratings at all
                         bridge_condition = case_when(lowest_condition_rating >= 7 ~ 'Good',
                                                      lowest_condition_rating <= 4 ~ 'Poor',
                                                      lowest_condition_rating == 5 | 
                                                      lowest_condition_rating == 6 ~ 'Fair'),
    
    # work proposed to be done
                         work_proposed = case_when(WORK_PROPOSED_075A == 31 ~ 'Replacement of bridge or other structure because of substandard load carrying capacity or substandard bridge roadway geometry',
                                                   WORK_PROPOSED_075A == 32 ~ 'Replacement of bridge or other structure because of relocation of road',
                                                   WORK_PROPOSED_075A == 33 ~ 'Widening of existing bridge or other major structure without deck rehabilitation or replacement: includes culvert lengthening',
                                                   WORK_PROPOSED_075A == 34 ~ 'Widening of existing bridge with deck rehabilitation or replacement',
                                                   WORK_PROPOSED_075A == 35 ~ 'Bridge rehabilitation because of general structure deterioration or inadquate strength',
                                                   WORK_PROPOSED_075A == 36 ~ 'Bridge deck rehabilitation with only incidental widening',
                                                   WORK_PROPOSED_075A == 37 ~ 'Bridge deck replacement with only incidental widening',
                                                   WORK_PROPOSED_075A == 38 ~ 'Other structural work, including hydraulic replacements',
                                                   TRUE ~ 'no work proposed'), 
                         
    # who will do the work? contractor or owner?  
                        work_responsibility = case_when(WORK_DONE_BY_075B == 1 ~ 'Contract',
                                                        WORK_DONE_BY_075B == 2 ~ 'Owner',
                                                        TRUE ~ 'no work proposed'),
    
    # month and year of inspection: derived from 4-digit inspection date variable, DATE_OF_INSPECT_090
                        inspection_mo = str_extract(str_pad(as.character(DATE_OF_INSPECT_090), 4, pad = "0"), "^.{2}"), 
                        inspection_yr = str_sub(str_pad(as.character(DATE_OF_INSPECT_090), 4, pad = "0"), start = -2), 
                        
    # is bridge vulnerable to scour (deterioration if bridge goes over water)
                        scour_critical = case_when(SCOUR_CRITICAL_113 == '3' | 
                                                   SCOUR_CRITICAL_113 == '2' | 
                                                   SCOUR_CRITICAL_113 == '1' |
                                                   SCOUR_CRITICAL_113 == '0' ~ 'yes',
                                                   SCOUR_CRITICAL_113 == '6' ~ 'not yet evaluated',
                                                   SCOUR_CRITICAL_113 == '7' |
                                                   SCOUR_CRITICAL_113 == '8' |
                                                   SCOUR_CRITICAL_113 == '9' ~ 'no',
                                                   SCOUR_CRITICAL_113 == 'T' | 
                                                   SCOUR_CRITICAL_113 == 'U' ~ 'unknown or low risk',
                                                   SCOUR_CRITICAL_113 == 'N' ~ 'not applicable'), 
                         
    # what's the kind of route carried
                         route_type = case_when(ROUTE_PREFIX_005B == 1 ~ 'Interstate highway', 
                                                ROUTE_PREFIX_005B == 2 ~ 'U.S. Numbered highway',
                                                ROUTE_PREFIX_005B == 3 ~ 'State highway', 
                                                ROUTE_PREFIX_005B == 4 ~ 'County highway',
                                                ROUTE_PREFIX_005B == 5 ~ 'City street',
                                                ROUTE_PREFIX_005B == 6 ~ 'Federal lands road',
                                                ROUTE_PREFIX_005B == 7 ~ 'State lands road', 
                                                ROUTE_PREFIX_005B == 8 ~ 'Other'),
                         
    # what's the service level for the road                    
                         service_level = case_when(SERVICE_LEVEL_005C == 0 ~ 'other', 
                                                   SERVICE_LEVEL_005C == 1 ~ 'mainline',
                                                   SERVICE_LEVEL_005C == 2 ~ 'alternate',
                                                   SERVICE_LEVEL_005C == 3 ~ 'bypass', 
                                                   SERVICE_LEVEL_005C == 4 ~ 'spur', 
                                                   SERVICE_LEVEL_005C == 6 ~ 'business', 
                                                   SERVICE_LEVEL_005C == 7 ~ 'ramp, wye, connector, etc.',
                                                   SERVICE_LEVEL_005C == 8 ~ 'service and/or unclassified frontage road'))
    
  df <- merge(df, fips %>% filter(`Summary Level` == '050') %>% # https://www.census.gov/programs-surveys/popest/guidance-geographies/terms-and-definitions.html
                  select(combinedfips, `Area Name (including legal/statistical area description)`),
                  by = 'combinedfips', all.x = T)
    
  df$location <- trimws(str_replace_all(df$location, "'", ""))
  df$intersects <- trimws(str_replace_all(df$intersects, "'", ""))
  df$carries <- trimws(str_replace_all(df$carries, "'", ""))
    
  df <- df %>% rename(county = `Area Name (including legal/statistical area description)`) %>% 
                        select(STRUCTURE_NUMBER_008,
                        county,
                        combinedfips,
                        location, 
                        intersects,
                        carries,
                        route_type,
                        service_level,
                        combinedfips,
                        responsibility,
                        owner,
                        op_status,
                        struct_deficient,
                        struct_deficient_old,
                        bridge_condition,
                        SUFFICIENCY_RATING,
                        lowest_condition_rating,
                        total_poor_conditions,
                        deck_condition, 
                        superstruct_condition, 
                        substruct_condition, 
                        culvert_condition,
                        deck_condition_poor, 
                        superstruct_condition_poor, 
                        substruct_condition_poor, 
                        culvert_condition_poor,
                        avg_daily_traffic, 
                        avg_daily_traffic_yr, 
                        avg_daily_truck_traffic_perc,
                        work_proposed,
                        work_responsibility,
                        inspection_mo,
                        inspection_yr,
                        yr_reconstructed, 
                        yr_built, 
                        inspection_freq,
                        bridge_improve_cost,
                        roadway_improve_cost,
                        total_improve_cost, 
                        yr_cost_estimate,
                        avg_daily_traffic_future, 
                        avg_daily_traffic_future_yr,
                        scour_critical,
                        national_register, 
                        LAT_016, # note latitude and longitude are provided in degrees, minutes, seconds
                        LONG_017,
                        SCOUR_CRITICAL_113, 
                        COUNTY_CODE_003, structural_eval, waterway_eval)
        
  return(df)
    
    }

Note the below function will return several error messages, for example: `NAs introduced by coercion”Warning message in evalq(as.numeric(SUPERSTRUCTURE_COND_059), <environment>):` and `no non-missing arguments to min; returning Inf”Warning message in min(deck_condition, superstruct_condition, substruct_condition`. These are due to the fact that structures that do not have a condition rating for a particular component will be labeled `NA` and, for the purposes of our analysis, can be ignored. We have set `options(warn = -1)` to suppress these warnings.

In [5]:
options(warn = -1)

data.2017.clean <- process(data.2017)
data.2007.clean <- process(data.2007)

## Write out the processed data files to the output folder

We have exported the 2007 and 2017 dataframes to the [feather format](https://blog.rstudio.com/2016/03/29/feather/),  which loads faster and can be opened in R and Python while preserving column types. They can also be exported as CSV files via `write_csv()`.

In [6]:
write_feather(data.2017.clean, 'output/data_2017_clean.feather')
write_feather(data.2007.clean, 'output/data_2007_clean.feather')

# write_csv(data.2017.clean, 'output/data_2017_clean.csv')
# write_csv(data.2007.clean, 'output/data_2007_clean.csv')